In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,train_test_split

# import data

In [22]:
pd.set_option('max_columns',50)

In [34]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [35]:
train.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,2,5,4,5,5,3,5,5.0,1,2,2,2,4,5,4,1,3,3,1,1,1.0,4.0,2.0,3.0,5,3,4,5,5,4,3,4,5653753.0
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,2,5,5,5,5,1,5,5.0,0,0,0,0,0,3,2,1,3,2,0,0,0.0,0.0,3.0,3.0,0,0,0,0,0,0,0,0,6923131.0
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,3,5,5,5,5,2,5,5.0,0,0,0,0,0,1,1,1,1,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,2055379.0
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,4,10,8,10,10,8,10,7.5,6,4,9,3,12,20,12,6,1,10,2,2,2.5,2.5,2.5,7.5,25,12,10,6,18,12,12,6,2675511.0
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,2,5,5,5,5,2,5,5.0,2,1,2,1,4,2,2,1,2,1,2,3,3.0,5.0,1.0,3.0,5,1,3,2,3,4,3,3,4316715.0


# preproccessing

In [32]:
#train&test preproccess together
train['train'] = 1
test['train'] = 0
datax = pd.concat([train.drop('revenue',axis=1),test],axis=0)
datay = train['revenue']

In [33]:
#date
datax['Open Date'] = pd.to_datetime(datax['Open Date'])

datax['year'] = datax['Open Date'].dt.year
datax['month'] = datax['Open Date'].dt.month
datax['day'] = datax['Open Date'].dt.day
datax['weekday'] = datax['Open Date'].dt.weekday

datax.drop('Open Date',axis=1,inplace=True)

In [7]:
#categorical to numbers
datax['City Group'] = datax['City Group'].map({'Other':0,'Big Cities':1})

In [8]:
#categorical to onehot
datax = pd.get_dummies(datax,columns=['Type'])

In [9]:
#categorical to numbers
datax['City'] = LabelEncoder().fit_transform(datax['City'])

In [10]:
datax.head()

,Id,City,City Group,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,train,year,month,day,weekday,Type_DT,Type_FC,Type_IL,Type_MB
0,0,60,1,4,5.0,4.0,4.0,2,2,5,4,5,5,3,5,5.0,1,2,2,2,4,5,4,1,3,3,1,1,1.0,4.0,2.0,3.0,5,3,4,5,5,4,3,4,1,1999,7,17,5,0,0,1,0
1,1,4,1,4,5.0,4.0,4.0,1,2,5,5,5,5,1,5,5.0,0,0,0,0,0,3,2,1,3,2,0,0,0.0,0.0,3.0,3.0,0,0,0,0,0,0,0,0,1,2008,2,14,3,0,1,0,0
2,2,14,0,2,4.0,2.0,5.0,2,3,5,5,5,5,2,5,5.0,0,0,0,0,0,1,1,1,1,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,1,2013,3,9,5,0,0,1,0
3,3,52,0,6,4.5,6.0,6.0,4,4,10,8,10,10,8,10,7.5,6,4,9,3,12,20,12,6,1,10,2,2,2.5,2.5,2.5,7.5,25,12,10,6,18,12,12,6,1,2012,2,2,3,0,0,1,0
4,4,21,0,3,4.0,3.0,4.0,2,2,5,5,5,5,2,5,5.0,2,1,2,1,4,2,2,1,2,1,2,3,3.0,5.0,1.0,3.0,5,1,3,2,3,4,3,3,1,2009,5,9,5,0,0,1,0


In [11]:
#train/test
train = datax[datax['train']==1].drop('train',axis=1)
test = datax[datax['train']==0].drop('train',axis=1)
train_x,test_x,train_y,test_y = train_test_split(train,datay,test_size=0.2,random_state=0)

# create model

In [12]:
import lightgbm as lgb

In [18]:
model_jmda = lgb.LGBMRegressor(n_estimators=84,max_depth=5)
model_jmda.fit(train_x,train_y,categorical_feature=['City'],eval_set=(test_x,test_y),eval_metric=['rmse'])

[1]	valid_0's rmse: 3.32602e+06	valid_0's l2: 1.10624e+13
[2]	valid_0's rmse: 3.32339e+06	valid_0's l2: 1.10449e+13
[3]	valid_0's rmse: 3.2996e+06	valid_0's l2: 1.08874e+13
[4]	valid_0's rmse: 3.30161e+06	valid_0's l2: 1.09007e+13
[5]	valid_0's rmse: 3.28456e+06	valid_0's l2: 1.07884e+13
[6]	valid_0's rmse: 3.28341e+06	valid_0's l2: 1.07808e+13
[7]	valid_0's rmse: 3.27085e+06	valid_0's l2: 1.06984e+13
[8]	valid_0's rmse: 3.27127e+06	valid_0's l2: 1.07012e+13
[9]	valid_0's rmse: 3.2576e+06	valid_0's l2: 1.06119e+13
[10]	valid_0's rmse: 3.26159e+06	valid_0's l2: 1.0638e+13
[11]	valid_0's rmse: 3.26294e+06	valid_0's l2: 1.06468e+13
[12]	valid_0's rmse: 3.26425e+06	valid_0's l2: 1.06553e+13
[13]	valid_0's rmse: 3.25766e+06	valid_0's l2: 1.06123e+13
[14]	valid_0's rmse: 3.24209e+06	valid_0's l2: 1.05111e+13
[15]	valid_0's rmse: 3.25185e+06	valid_0's l2: 1.05745e+13
[16]	valid_0's rmse: 3.23299e+06	valid_0's l2: 1.04522e+13
[17]	valid_0's rmse: 3.22539e+06	valid_0's l2: 1.04032e+13
[18]	vali

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=1, importance_type='split', learning_rate=0.1,
              max_depth=5, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=84, n_jobs=-1, num_leaves=31,
              objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

In [19]:
#dirty fitting
model_jmda = lgb.LGBMRegressor(n_estimators=84,max_depth=5)
model_jmda.fit(train,datay,categorical_feature=['City'],eval_set=(test_x,test_y),eval_metric=['rmse'])

[1]	valid_0's rmse: 3.25413e+06	valid_0's l2: 1.05894e+13
[2]	valid_0's rmse: 3.21137e+06	valid_0's l2: 1.03129e+13
[3]	valid_0's rmse: 3.1622e+06	valid_0's l2: 9.99948e+12
[4]	valid_0's rmse: 3.12647e+06	valid_0's l2: 9.77479e+12
[5]	valid_0's rmse: 3.08439e+06	valid_0's l2: 9.51347e+12
[6]	valid_0's rmse: 3.05407e+06	valid_0's l2: 9.32732e+12
[7]	valid_0's rmse: 3.02954e+06	valid_0's l2: 9.17813e+12
[8]	valid_0's rmse: 3.0062e+06	valid_0's l2: 9.03724e+12
[9]	valid_0's rmse: 2.97337e+06	valid_0's l2: 8.84096e+12
[10]	valid_0's rmse: 2.95359e+06	valid_0's l2: 8.72368e+12
[11]	valid_0's rmse: 2.93369e+06	valid_0's l2: 8.60656e+12
[12]	valid_0's rmse: 2.90995e+06	valid_0's l2: 8.46781e+12
[13]	valid_0's rmse: 2.88648e+06	valid_0's l2: 8.33176e+12
[14]	valid_0's rmse: 2.85203e+06	valid_0's l2: 8.13409e+12
[15]	valid_0's rmse: 2.82507e+06	valid_0's l2: 7.98104e+12
[16]	valid_0's rmse: 2.80529e+06	valid_0's l2: 7.86967e+12
[17]	valid_0's rmse: 2.78638e+06	valid_0's l2: 7.76393e+12
[18]	val

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=5,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=84, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

# submission

In [ ]:
pred = model_jmda.predict(test)

In [ ]:
sub = pd.read_csv('./sampleSubmission.csv')

In [ ]:
sub['Prediction'] = pred
sub.to_csv('./sampleSubmission.csv',index=False)